# Fill in Repricer.com inventory template

In [103]:
import pandas as pd
import numpy as np
import pyisbn #module for converting ISBN-13 to ISBN-10 and vice versa


invhealth = pd.read_csv('data/inventory_health_aug19.csv',engine='python')
FBAinv = pd.read_csv('data/FBA_inventory_aug19.csv',engine='python')
inv = pd.read_csv('data/inventory_spreadsheet_aug19.csv')
products = pd.read_csv('data/CDGD_sourcing_pricing_aug20.csv')
template = pd.read_csv('data/69807_1.csv')


In [104]:
products.columns

Index(['Title', 'updated 2020?', 'Cover Art', 'ISBN-13', 'ASIN', 'DealOz Link',
       'Amazon Link', 'New ed exists?', 'Seasonality', 'Season',
       'Seasonality notes', 'Trend notes', 'Popularity notes',
       'Approved buy price', 'Approved list price',
       'List price MUST not go below:', 'Notes', 'Keepa'],
      dtype='object')

In [110]:
#set up fixed columns
template['price_max'] = 299.99
template['price_min'] = 9.99
template['repricer_name'] = 'PRIME comp 0.01 + kickstart'

In [88]:
template.columns

Index(['sku', 'marketplace', 'merchant_id', 'price_min', 'price_max',
       'unit_cost', 'unit_currency', 'shipping_cost', 'shipping_currency',
       'pickpack_cost', 'pickpack_currency', 'rrp_price', 'rrp_currency',
       'vat_rate', 'fee_listing', 'fba_fee', 'fba_currency', 'repricer_name',
       'velocity_repricer_name', 'estimated_seller_count',
       'lowest_landed_price', 'my_price', 'my_shipping', 'fba', 'asin',
       'open_date', 'last_sale', 'quantity'],
      dtype='object')

In [111]:
# pick out only active SKUs
template.reset_index() #restore to numerical if messed up
active_skus = list(set(FBAinv['sku']).intersection(set(template['sku'])))

template.set_index('sku', inplace=True)
template = template.loc[active_skus]
template = template.reset_index()


In [112]:
# Load min/max prices, if available, into template and save
for asin in template['asin']: 
    min_price = products.loc[products['ASIN'] == asin]['Approved list price'].to_list()
    if min_price != []:
        if min_price[0] > 9.99:
            template.at[template.index[template['asin']==asin], 'price_min'] = min_price[0]
        else: 
            template.at[template.index[template['asin']==asin], 'price_min'] = 'idklol'
    else: 
        template.at[template.index[template['asin']==asin], 'price_min'] = 'idklol'

In [113]:
template

,sku,marketplace,merchant_id,price_min,price_max,unit_cost,unit_currency,shipping_cost,shipping_currency,pickpack_cost,...,velocity_repricer_name,estimated_seller_count,lowest_landed_price,my_price,my_shipping,fba,asin,open_date,last_sale,quantity
0,CP_July25-001,AUS,AY1C08ZS4IKQC,140,299.99,0,USD,0,USD,0,...,Off,56,98.07,104.99,0.0,Yes,1285740629,2020-07-25,2020-08-06 03:48:12,0
1,CP 5-27-20-007,AUS,AY1C08ZS4IKQC,80,299.99,0,USD,0,USD,0,...,Off,54,85.09,179.99,0.0,Yes,1118156595,2020-05-27,NaN,2
2,X1-6L9F-MMOE,AUS,AY1C08ZS4IKQC,idklol,299.99,0,USD,0,USD,0,...,Off,4,221.65,63.60,0.0,Yes,0134146778,2019-04-04,NaN,1
3,cp_Apr18-018,AUS,AY1C08ZS4IKQC,170,299.99,0,USD,0,USD,0,...,Off,0,0.00,169.99,0.0,Yes,0205052177,2020-05-05,2020-08-05 21:18:48,0
4,Christine 05/30-033,AUS,AY1C08ZS4IKQC,idklol,299.99,0,USD,0,USD,0,...,Off,0,0.00,9.99,0.0,Yes,1305112121,2019-06-08,2020-08-17 16:22:44,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,r_may22-007,AUS,AY1C08ZS4IKQC,idklol,299.99,0,USD,0,USD,0,...,Off,4,98.99,25.95,0.0,Yes,1451130600,2019-05-22,NaN,2
514,CP_June07-009,AUS,AY1C08ZS4IKQC,70,299.99,0,USD,0,USD,0,...,Off,27,67.00,72.81,0.0,Yes,1111827052,2020-06-07,2020-08-16 16:37:59,0
515,CP_June172020-021,AUS,AY1C08ZS4IKQC,170,299.99,0,USD,0,USD,0,...,Off,19,144.97,169.99,0.0,Yes,1337275344,2020-06-18,NaN,1
516,cp_Apr18-036,AUS,AY1C08ZS4IKQC,110,299.99,0,USD,0,USD,0,...,Off,27,48.98,82.37,0.0,Yes,0078028221,2020-05-05,NaN,1


In [129]:
# Fix idklol to $10+1.3*avg_buy_price

for asin in template.loc[template['price_min'] == 'idklol']['asin']:
    #convert to isbn
    try:
        isbn = products.loc[products['ASIN'] == asin]['ISBN-13'].to_list()[0]
        avg_buy_cost = inv.loc[inv['ISBN'] == isbn]['Item Cost']
        if len(avg_buy_cost)>0:
            avg_buy_cost = round(np.mean(avg_buy_cost),2)
            min_price = (avg_buy_cost+10)*1.3
        else:
            min_price = 'check_me'
    except:
        min_price = 'check_me'
   
    template.at[template.index[template['asin']==asin], 'price_min'] = min_price


In [130]:
template

,sku,marketplace,merchant_id,price_min,price_max,unit_cost,unit_currency,shipping_cost,shipping_currency,pickpack_cost,...,velocity_repricer_name,estimated_seller_count,lowest_landed_price,my_price,my_shipping,fba,asin,open_date,last_sale,quantity
0,CP_July25-001,AUS,AY1C08ZS4IKQC,140,299.99,0,USD,0,USD,0,...,Off,56,98.07,104.99,0.0,Yes,1285740629,2020-07-25,2020-08-06 03:48:12,0
1,CP 5-27-20-007,AUS,AY1C08ZS4IKQC,80,299.99,0,USD,0,USD,0,...,Off,54,85.09,179.99,0.0,Yes,1118156595,2020-05-27,NaN,2
2,X1-6L9F-MMOE,AUS,AY1C08ZS4IKQC,check_me,299.99,0,USD,0,USD,0,...,Off,4,221.65,63.60,0.0,Yes,0134146778,2019-04-04,NaN,1
3,cp_Apr18-018,AUS,AY1C08ZS4IKQC,170,299.99,0,USD,0,USD,0,...,Off,0,0.00,169.99,0.0,Yes,0205052177,2020-05-05,2020-08-05 21:18:48,0
4,Christine 05/30-033,AUS,AY1C08ZS4IKQC,63.18,299.99,0,USD,0,USD,0,...,Off,0,0.00,9.99,0.0,Yes,1305112121,2019-06-08,2020-08-17 16:22:44,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,r_may22-007,AUS,AY1C08ZS4IKQC,check_me,299.99,0,USD,0,USD,0,...,Off,4,98.99,25.95,0.0,Yes,1451130600,2019-05-22,NaN,2
514,CP_June07-009,AUS,AY1C08ZS4IKQC,70,299.99,0,USD,0,USD,0,...,Off,27,67.00,72.81,0.0,Yes,1111827052,2020-06-07,2020-08-16 16:37:59,0
515,CP_June172020-021,AUS,AY1C08ZS4IKQC,170,299.99,0,USD,0,USD,0,...,Off,19,144.97,169.99,0.0,Yes,1337275344,2020-06-18,NaN,1
516,cp_Apr18-036,AUS,AY1C08ZS4IKQC,110,299.99,0,USD,0,USD,0,...,Off,27,48.98,82.37,0.0,Yes,0078028221,2020-05-05,NaN,1


In [131]:
template.to_csv(r'C:\Users\Lyra\Documents\GitHub\CDGD\data\template_filledin_aug20.csv', index = False)